<a href="https://colab.research.google.com/github/Leen020/CSE4078S25_Grp1/blob/main/delivery3/fine_tuned_model_gemma3-1b-CSE4078S25_Grp1-r16-tr-ner-lr5e-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Data Prep and Fetching from Github

In [3]:
!git clone "https://github.com/Leen020/CSE4078S25_Grp1.git"

fatal: destination path 'CSE4078S25_Grp1' already exists and is not an empty directory.


In [4]:
%cd CSE4078S25_Grp1

/content/CSE4078S25_Grp1


In [5]:
from datasets import Dataset
import json

# Load your Alpaca-style dataset
with open("delivery3/CSE4078S25_Grp1_train_AlpacaStyle.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open("delivery3/CSE4078S25_Grp1_testset_AlpacaStyle.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Convert to HuggingFace dataset
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)


### Unsloth

In [6]:
# %%
!pip install transformers datasets bitsandbytes unsloth huggingface_hub

  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.8.5.post1 requires ray[cgraph]!=2.44.*,>=2.43.0, which is not installed.
vllm 0.8.5.post1 requires compressed-tensors==0.9.3, but you have compressed-tensors 0.9.4 which is incompatible.
vllm 0.8.5.post1 requires numba==0.61.2; python_version > "3.9", but you have numba 0.60.0 which is incompatible.
vllm 0.8.5.post1 requires opentelemetry-api<1.27.0,>=1.26.0, but you have opentelemetry-api 1.33.1 which is incompatible.
vllm 0.8.5.post1 requires opentelemetry-exporter-otlp<1.27.0,>=1.26.0, but you have opentelemetry-ex

Load up `Gemma 3 1B Instruct`, and set parameters

In [7]:
from unsloth import FastModel
import torch
max_seq_length = 2048

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-18 12:33:44 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-18 12:33:44 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.6: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [8]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


Convert to Unsloth's required format! and see the first row:

Use the tokenizer's chat template to format the input and output into a single text field

In [9]:
def format_chat(instance, tokenizer):
    # Construct the messages list for the chat template
    messages = [
        {"role": "system", "content": instance["instruction"]},
        {"role": "user", "content": instance["input"]},
        # Include the output as the assistant's response for training
        {"role": "assistant", "content": instance["output"]},
    ]
    # Apply the chat template to get the combined text
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,  # We want the string, not the token IDs
        add_generation_prompt=False # Do not add generation prompt for training
    )
    # Return a dictionary with a single "text" field
    return {"text": formatted_text}

# Apply the formatting function to the datasets
# Use batched=True for potentially faster processing, but not strictly necessary for correctness
train_dataset = train_dataset.map(lambda x: format_chat(x, tokenizer))
test_dataset = test_dataset.map(lambda x: format_chat(x, tokenizer))


# Print the first example to verify the format
print(train_dataset[0])
print(train_dataset[0]['text'])

Map:   0%|          | 0/31248 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'instruction': 'Aşağıdaki cümledeki kelimelere doğru NER etiketlerini ekleyin.', 'input': 'Albüm satış rakamlarıyla dikkat çekmiştir .', 'output': 'Albüm: O, satış: O, rakamlarıyla: O, dikkat: O, çekmiştir: O, .: O', 'text': '<bos><start_of_turn>user\nAşağıdaki cümledeki kelimelere doğru NER etiketlerini ekleyin.\n\nAlbüm satış rakamlarıyla dikkat çekmiştir .<end_of_turn>\n<start_of_turn>model\nAlbüm: O, satış: O, rakamlarıyla: O, dikkat: O, çekmiştir: O, .: O<end_of_turn>\n'}
<bos><start_of_turn>user
Aşağıdaki cümledeki kelimelere doğru NER etiketlerini ekleyin.

Albüm satış rakamlarıyla dikkat çekmiştir .<end_of_turn>
<start_of_turn>model
Albüm: O, satış: O, rakamlarıyla: O, dikkat: O, çekmiştir: O, .: O<end_of_turn>



<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [10]:
# Install the trl library
!pip install trl

In [11]:
max_prompt_length = 256 # This might not be directly used by SFTTrainer with packing=False

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-5,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch",
    logging_steps = 10,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [13]:
from transformers import DataCollatorForLanguageModeling
from trl import SFTTrainer
from transformers import TrainingArguments

# Define training arguments suitable for SFTTrainer
training_args = TrainingArguments(
    output_dir="outputs",
    num_train_epochs=1,
    max_steps=50,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    save_steps=50,
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.0,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
    fp16=False,
)

# Check the type of the 'text' field and print the first few characters
print(f"Type of train_dataset[0]['text']: {type(train_dataset[0]['text'])}")
print(f"First 200 characters of train_dataset[0]['text']:\n{train_dataset[0]['text'][:200]}")


# Re-initialize SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = training_args,
    packing = False,
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# Start training
trainer.train()

Type of train_dataset[0]['text']: <class 'str'>
First 200 characters of train_dataset[0]['text']:
<bos><start_of_turn>user
Aşağıdaki cümledeki kelimelere doğru NER etiketlerini ekleyin.

Albüm satış rakamlarıyla dikkat çekmiştir .<end_of_turn>
<start_of_turn>model
Albüm: O, satış: O, rakamlarıyla:
Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/31248 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 31,248 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 13,045,760/1,012,931,712 (1.29% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,3.363600
20,2.526500
30,1.922200
40,1.786600
50,1.689200


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=50, training_loss=2.257635612487793, metrics={'train_runtime': 129.6153, 'train_samples_per_second': 6.172, 'train_steps_per_second': 0.386, 'total_flos': 848591536195584.0, 'train_loss': 2.257635612487793})

In [14]:
# Save merged model (not just adapters or LoRA weights)
merged_output_dir = "fine_tuned_models/fine_tuned_model_gemma3-1b-CSE4078S25_Grp1-r16-tr-ner-lr5e-5"
trainer.model.save_pretrained(merged_output_dir)
trainer.tokenizer.save_pretrained(merged_output_dir)

print(f"Model and tokenizer saved to {merged_output_dir}")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Model and tokenizer saved to fine_tuned_models/fine_tuned_model_gemma3-1b-CSE4078S25_Grp1-r16-tr-ner-lr5e-5


In [18]:
pip install evaluate bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00


In [20]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ec849ee33ad5f03407f8e94c490ad32b1acf30307f075690c87691f5068fb1e2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import torch
from tqdm import tqdm
from datasets import Dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

# Convert back to string
def format_chat_for_generation(instance, tokenizer):
    messages = [
        {"role": "system", "content": instance["instruction"]},
        {"role": "user", "content": instance["input"]}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Run evaluation
predictions = []
references = []

model.eval()

for instance in tqdm(test_dataset):
    prompt = format_chat_for_generation(instance, tokenizer)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False
        )
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    # Store predictions and references
    predictions.append(generated)
    references.append(instance["output"].strip())

# Compute Metrics
rouge_result = rouge.compute(predictions=predictions, references=references)
bleu_result = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="tr")  # Turkish

# Print results
print("\n--- Evaluation Results ---")
print("ROUGE-1:", rouge_result["rouge1"])
print("ROUGE-2:", rouge_result["rouge2"])
print("ROUGE-L:", rouge_result["rougeL"])
print("BLEU:", bleu_result["bleu"])
print("BERTScore (F1):", sum(bertscore_result["f1"]) / len(bertscore_result["f1"]))


  2%|▏         | 18/1000 [03:29<2:40:33,  9.81s/it]

<a name="Inference"></a>
### Inference
Now let's try the model we just trained!

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
